# One hot encoding using tdplyr and dplyr

This notebook illustrates how one can perform one-hot encoding on a data frame using regular dplyr verbs. It 
extends the concept to a remote tbl as well wherein dbplyr abstracts the underlyting SQL translations. Here 
I have demonstrated how it can be done with remote Teradata tbl's using tdplyr R package.

In [1]:
# Import necessary libraries
library(dplyr)
library(tdplyr)
library(vcd)
library(rlang)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Registered S3 methods overwritten by 'tdplyr':
  method                     from  
  db_analyze.Teradata        dbplyr
  sql_translate_env.Teradata dbplyr
Loading required package: grid


In [2]:
# One hot encoding function
one_hot_encode <- function(df, column) {
  if(is.null(column) || column == '' || is.na(column))
    stop('Column parameter cannot be NULL, NA or empty')
  if(!(inherits(df, "data.frame") || inherits(df, "tbl_teradata")))
    stop('Invalid class for the input to be encoded')
  if(!inherits(column, "character"))
    stop('Invalid class for the `column` parameter')
  if(length(column) > 1)
    stop('Only one column can be one-hot encoded at a time')
  if(!column %in% colnames(df))
    stop('Column not found in the input data')
  if(inherits(df, "data.frame") && !inherits_any(df[[column]], c("character", "factor")))
    stop('Provided column in data frame cannot be one-hot encoded')
  if(inherits(df, "tbl_teradata") && df$types[which(!!sym(column) == colnames(df))][[1]] != "varchar")
    stop('Provided column in tbl_teradata cannot be one-hot encoded')
  
  unique_val <- as.data.frame(df %>% distinct(!!sym(column)) %>% select(!!sym(column)))
  for (val in unique_val[[column]]){
    colname <- paste(column, val, sep = '_')
    df <- df %>% mutate(
      !!(sym(colname)) := if_else(!!sym(column) == val, 1L, 0L)
    )
  }
  return(df)
}

In [3]:
# Load data into memory
data('Arthritis')
df <- Arthritis
head(df)

ID,Treatment,Sex,Age,Improved
<int>,<fct>,<fct>,<int>,<ord>
57,Treated,Male,27,Some
46,Treated,Male,29,None
77,Treated,Male,30,None
17,Treated,Male,32,Marked
36,Treated,Male,46,Marked
23,Treated,Male,58,Marked


In [4]:
# Establish connection with the Teradata database
con <- td_create_context(host='*****', uid='*****', pwd='*****', dType='native')

In [5]:
ats <- copy_to(con, df, "arthritis", overwrite = TRUE)
ats

[tdplyr - (TDR_E1001)] Note: This will drop existing table and recreate it with new schema.


# Source:   table<arthritis> [?? x 5]
# Database: [Teradata 16.20.34.01] [Teradata Native Driver 16.20.0.18]
#   [ALICE@tdap1563t1.labs.teradata.com1025/ALICE]
      ID Treatment Sex      Age Improved
   <int> <chr>     <chr>  <int> <chr>   
 1    13 Treated   Female    67 Marked  
 2    61 Treated   Female    68 Some    
 3    38 Placebo   Female    32 None    
 4    78 Placebo   Female    54 Marked  
 5    19 Placebo   Female    63 Some    
 6    59 Treated   Female    59 Marked  
 7    36 Treated   Male      46 Marked  
 8    15 Placebo   Female    66 Some    
 9    40 Treated   Female    32 None    
10    80 Placebo   Female    23 None    
# … with more rows

In [7]:
# Perform one-hot encoding on the remote tbl's 'Sex' and 'Improved' column
ats1 <- one_hot_encode(ats, 'Sex')
ats2 <- one_hot_encode(ats1, "Improved")
ats2

# Source:   lazy query [?? x 10]
# Database: [Teradata 16.20.34.01] [Teradata Native Driver 16.20.0.18]
#   [ALICE@tdap1563t1.labs.teradata.com1025/ALICE]
      ID Treatment Sex     Age Improved Sex_Female Sex_Male Improved_Marked
   <int> <chr>     <chr> <int> <chr>         <int>    <int>           <int>
 1    13 Treated   Fema…    67 Marked            1        0               1
 2    61 Treated   Fema…    68 Some              1        0               0
 3    38 Placebo   Fema…    32 None              1        0               0
 4    78 Placebo   Fema…    54 Marked            1        0               1
 5    19 Placebo   Fema…    63 Some              1        0               0
 6    59 Treated   Fema…    59 Marked            1        0               1
 7    36 Treated   Male     46 Marked            0        1               1
 8    15 Placebo   Fema…    66 Some              1        0               0
 9    40 Treated   Fema…    32 None              1        0               0
10    80 

In [8]:
# To view the corresponding SQL queries of the remote tbl's
ats2 %>% show_query()

<SQL>
SELECT "ID", "Treatment", "Sex", "Age", "Improved", "Sex_Female", "Sex_Male", "Improved_Marked", "Improved_Some", CASE WHEN ("Improved" = 'None') THEN (1) WHEN NOT("Improved" = 'None') THEN (0) END AS "Improved_None"
FROM (SELECT "ID", "Treatment", "Sex", "Age", "Improved", "Sex_Female", "Sex_Male", "Improved_Marked", CASE WHEN ("Improved" = 'Some') THEN (1) WHEN NOT("Improved" = 'Some') THEN (0) END AS "Improved_Some"
FROM (SELECT "ID", "Treatment", "Sex", "Age", "Improved", "Sex_Female", "Sex_Male", CASE WHEN ("Improved" = 'Marked') THEN (1) WHEN NOT("Improved" = 'Marked') THEN (0) END AS "Improved_Marked"
FROM (SELECT "ID", "Treatment", "Sex", "Age", "Improved", "Sex_Female", CASE WHEN ("Sex" = 'Male') THEN (1) WHEN NOT("Sex" = 'Male') THEN (0) END AS "Sex_Male"
FROM (SELECT "ID", "Treatment", "Sex", "Age", "Improved", CASE WHEN ("Sex" = 'Female') THEN (1) WHEN NOT("Sex" = 'Female') THEN (0) END AS "Sex_Female"
FROM "arthritis") "dbplyr_021") "dbplyr_022") "dbplyr_023") "dbply

In [9]:
# One hot encoding a column in a data frame
a1 <- one_hot_encode(df, 'Treatment')
a2 <- one_hot_encode(a1, 'Sex')
head(a2)

ID,Treatment,Sex,Age,Improved,Treatment_Treated,Treatment_Placebo,Sex_Male,Sex_Female
<int>,<fct>,<fct>,<int>,<ord>,<int>,<int>,<int>,<int>
57,Treated,Male,27,Some,1,0,1,0
46,Treated,Male,29,None,1,0,1,0
77,Treated,Male,30,None,1,0,1,0
17,Treated,Male,32,Marked,1,0,1,0
36,Treated,Male,46,Marked,1,0,1,0
23,Treated,Male,58,Marked,1,0,1,0
